In [4]:
from functions import *
from models import *

2024-10-31 14:53:55.733347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 14:53:55.754480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 14:53:55.760673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 14:53:55.777458: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 14:53:56.997353: W tensorflow/compiler/tf2

In [5]:
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import time
from datetime import datetime

## Load data

In [6]:
# Set variables
source_folder = '../data/external/raster_masks'
rivers = {}
source_path = '../data/preprocessed/'
data_paths = ['lst','wt','ndvi','slope', 'discharge','masked']#,'slope', 'discharge']#, 'ndvi', 'wt', 'masked','discharge', 'slope']#, 'wt_interpolated']
dir_paths = [os.path.join(source_path,p) for p in data_paths]
all_dir_paths = {k:[] for k in data_paths}    
total_data = {}
total_times = {}
complete_rivers = []
filter_river = None
W=256

# Load rivers
for subdir, dirs, files in os.walk(source_folder):
    for i,file in enumerate(files):
        r,m = load_raster(os.path.join(subdir, file), False)
        name = file.split('.')[0].split('bw_')[-1]
        rivers[name] = r

# Load input paths
for i,dir_p in enumerate(dir_paths):
    for subdir, dirs,files in os.walk(dir_p):
        if subdir != dir_p and not subdir.endswith('masked') and not subdir.endswith('.ipynb_checkpoints'): 
            all_dir_paths[data_paths[i]].append(subdir)
        elif subdir.endswith('masked'):
            all_dir_paths['masked'].append(subdir)

# Load input data
for k,v in all_dir_paths.items():
    if filter_river != None:
        v = [v[i] for i in filter_river]
    
    if k != 'discharge' and k != 'slope':
        if k == 'lst' or k == 'masked':
            list_rgb = [True]*len(v)
        else:
            list_rgb = [False]*len(v)
            
        data, times = load_data(v,W,list_rgb)
        if k!='masked':
            labels = []
            for ki,value in data.items():
                labels+=[ki.split('/')[-1]]*len(value)
        
        filtered = [arr for arr in data.values() if arr.size > 0]

        total_data[k] = np.concatenate(filtered, axis=0)
        total_times[k] = times
        print(k,':' ,total_data[k].shape)

    elif k == 'discharge' or k == 'slope':
        total = []
        for p in v:
            for file in os.listdir(p):
                file_path = os.path.join(p, file)
                r,m = load_raster(file_path, False)
                var = resize_image(r, W,W)
                img_river = labels.count(p.split("/")[-1])
                var_input = np.tile(var, (img_river, 1, 1))
                total.append(var_input)
        
        total_data[k] = np.concatenate(total, axis=0)
        print(k,':' ,total_data[k].shape)

# Hot encoding
encoder = OneHotEncoder(sparse_output=False)
river_encoded = encoder.fit_transform(np.array(labels).reshape(-1, 1))
data_targets = total_data['wt']
results = {'MAE':0,'MSE':0,'RMSE':0,'R²':0,'MAPE (%)':0,'MSE sample-wise':0}


lst : (147, 256, 256, 3)
wt : (147, 256, 256)
ndvi : (147, 256, 256)
slope : (147, 256, 256)
discharge : (147, 256, 256)
masked : (147, 256, 256, 3)


## Do experiment

In [26]:
W = 256
filter_river = None#[3,11,12]
inputs = ['lst','ndvi','discharge', 'slope']#['ndvi','discharge', 'slope']
conditioned = False
batch_size = 16
epochs = 10
model_name = "img_wise_CNN_improved" #img_wise_CNN, UNet, transfer_learning_VGG16, CNN, img_2_img
stratified = False
physics_guided = True

#### Choose model

In [8]:
def get_results(test_target, test_prediction, rivers, labels, test_index):
    mean_results = {k:[] for k in results.keys()}
    # Loop through each sample and compute the MSE for that sample
    for i in range(test_target.shape[0]):
        # Flatten the true and predicted values for this sample
        riv = rivers[labels[test_index[i]]].flatten()
        y_true_flatten = test_target[i].flatten()
        y_true_mask = y_true_flatten[riv != 0]
        y_pred_flatten = test_prediction[i].flatten()
        y_pred_mask = y_pred_flatten[riv != 0]
        # Calculate metrics
        res = evaluate_model(y_true_mask, y_pred_mask)
        for k,v in res.items():
            mean_results[k].append(v)
    for key in mean_results:
        mean_results[key] = np.mean(mean_results[key])
    return mean_results

In [27]:
# Choose inputs
inputs_d = [total_data[inp] for inp in inputs]
# List to store the processed additional images
expanded_images = []
# Expand dimensions for single-channel images, leave multi-channel images as they are
for img in inputs_d:
    if img.ndim == 3:  # Case where image is (n, 256, 256) (single-channel)
        expanded_images.append(np.expand_dims(img, axis=-1))  # Expand to add an extra channel
    elif img.ndim == 4:  # Case where image already has multiple channels (n, 256, 256, c)
        expanded_images.append(img)  # Leave the image as it is
# Concatenate all images along the last axis (channels)
combined_input = np.concatenate(expanded_images, axis=-1)
# The final combined input is stored in input_data
input_data = combined_input

### Split data

In [28]:
time_split = True
if time_split:
    train_ratio = 0.6
    val_ratio = 0.2
    test_ratio = 0.2
    
    # Calcular el tamaño de cada conjunto
    total_images = len(input_data)
    train_size = int(total_images * train_ratio)
    val_size = int(total_images * val_ratio)
    indices = np.arange(total_images)
    
    train_index = indices[:train_size]                       # Primeros índices para entrenamiento
    validation_index = indices[train_size:train_size + val_size]    # Siguientes índices para validación
    test_index = indices[train_size + val_size:]             # Últimos índices para prueba
   
elif stratified:
    train_index, validation_index, test_index = split_data_stratified(input_data, data_targets, labels)
else:
    train_index, validation_index, test_index = split_data(input_data, data_targets)
        
validation_input = input_data[validation_index, :] / 255.0  # Normalize inputs
validation_target = data_targets[validation_index, :]
validation_rivers = river_encoded[validation_index, :]
test_input = input_data[test_index, :] / 255.0  # Normalize inputs
test_target = data_targets[test_index, :]
test_rivers = river_encoded[test_index, :]
train_input = input_data[train_index, :] / 255.0  # Normalize inputs
train_target = data_targets[train_index, :]
train_rivers = river_encoded[train_index, :]
print(f"Train: {len(train_input)} imágenes, {train_input.shape}")
print(f"Val: {len(validation_input)} imágenes, {validation_input.shape}")
print(f"Test: {len(test_input)} imágenes, {test_input.shape}")
print(train_target.shape, validation_target.shape, test_target.shape)
    

Train: 88 imágenes, (88, 256, 256, 6)
Val: 29 imágenes, (29, 256, 256, 6)
Test: 30 imágenes, (30, 256, 256, 6)
(88, 256, 256) (29, 256, 256) (30, 256, 256)


#### Select inputs and model

In [29]:
if len(train_input.shape) == 3:
    input_shape = train_input.shape[1:]+(1,)
else:
    input_shape = train_input.shape[1:]


# Adapt input to condition
if conditioned:
    input_args = (input_shape, river_encoded.shape[1])
    model_input = (train_input, train_rivers)
    val_model_input = [validation_input, validation_rivers]
    test_model_input = [test_input, test_rivers]
else:
    input_args = input_shape
    model_input = train_input
    val_model_input = validation_input
    test_model_input = test_input


In [30]:
from tensorflow.keras import layers, models, regularizers

def build_simplified_cnn_model(input_shape):
    model = models.Sequential()

    # Capa 1: Convolucional + BatchNormalization + ReLU + Max Pooling + Dropout
    model.add(layers.Conv2D(16, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0005), input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))

    # Capa 2: Convolucional + BatchNormalization + ReLU + Max Pooling + Dropout
    model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))

    # Capa de aplanamiento
    model.add(layers.Flatten())

    # Capa densa + BatchNormalization + Dropout
    model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    # Capa de salida con activación lineal (para predicciones de temperatura)
    model.add(layers.Dense(256 * 256, activation='linear'))

    # Reshape de la salida a la forma (256, 256)
    model.add(layers.Reshape((256, 256)))

    return model



In [31]:
# Start model
start_time = time.time()
if model_name == "img_wise_CNN_improved":
    if conditioned:
        model = build_simplified_cnn_model_label(input_args[0], input_args[1])
    else:
        model = build_simplified_cnn_model(input_args)
elif model_name == 'CNN':
    model = build_cnn_model(input_args)
elif model_name == 'img_2_img':
    model = build_img_2_img_model(input_args)
elif model_name == 'UNet':
    model = build_unet(input_args)
elif model_name == 'transfer_learning_VGG16':
    train_input = train_input[:, :, :, :3]
    model = build_transfer_model((W, W, 3))

/home/ruth.parajo/miniconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Train model

In [32]:
print(f"Running experiment with model={model_name}, batch_size={batch_size}, epochs={epochs}")

# Train the model
if not physics_guided:
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    history = model.fit(model_input, train_target, batch_size=batch_size, epochs=epochs, validation_data=(val_model_input, validation_target))
else:
    dataset = tf.data.Dataset.from_tensor_slices((*model_input, train_target) if isinstance(model_input, tuple) else (model_input, train_target))

    dataset = dataset.batch(batch_size)
    optimizer = tf.keras.optimizers.Adam()
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        for batch in dataset:
            # Handle batch based on whether model_input is a tuple or a single dataset
            if isinstance(model_input, tuple):
                model_input_batch = batch[:-1]  # All except the last element (target_batch)
                target_batch = batch[-1]        # Last element is target_batch
            else:
                model_input_batch, target_batch = batch  # Direct unpacking for single dataset

            with tf.GradientTape() as tape:
                y_pred = model([*model_input_batch], training=True) if isinstance(model_input_batch, tuple) else model(model_input_batch, training=True)
                loss = conservation_energy_loss(target_batch, y_pred, model_input_batch, alpha=0.5, beta=0.5)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))


# Evaluate results
#validation_prediction = model.predict(val_model_input)
test_prediction = model.predict(test_model_input)

print('\nComputing result metrics...')
mean_results = get_results(test_target, test_prediction, rivers, labels, test_index)

# Get experiment data
end_time = time.time()
duration = round(end_time - start_time, 2)
current_date = datetime.now().strftime("%Y-%m-%d")
current_time = datetime.now().strftime("%H:%M:%S")


# Save model results
laabeel = 'label' if conditioned else 'no label'
var_inputs = '' if inputs == None else ', '.join(inputs)
variables = ', '.join([var_inputs, laabeel])
details = {'RMSE':mean_results['RMSE'],'Variables':variables,'Input': f'{len(np.unique(labels))} rivers', 'Output': 'wt', \
           'Resolution': W, 'nº samples': len(data_targets), 'Batch size': batch_size, 'Epochs': epochs, 'Date':current_date, \
           'Time':current_time, 'Duration': duration, 'Loss': 'Physics-guided'}

file_path = f"../results/{model_name}_results.xlsx"
save_excel(file_path, details, excel = 'Results')

mean_results['Model'] = model_name
file_path = f"../results/all_results.xlsx"
save_excel(file_path, mean_results, excel = 'Results')

print(f"Experiment {model_name} with batch_size={batch_size} and epochs={epochs} completed.\n")


Running experiment with model=img_wise_CNN_improved, batch_size=16, epochs=10
Epoch 1/10


W0000 00:00:1730384664.295927 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.297928 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.299789 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.301025 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.302298 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.303947 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.306160 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.307828 3347632 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1730384664.309497 3347632 gp

Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10


2024-10-31 15:24:30.527899: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step

Computing result metrics...
Experiment img_wise_CNN_improved with batch_size=16 and epochs=10 completed.



In [20]:
import pandas as pd
#model_name = 'img_wise_CNN'
pd.read_excel(f'../results/{model_name}_results.xlsx')

,RMSE,Variables,Input,Output,Resolution,nº samples,Batch size,Epochs,Date,Time,Duration,Loss
0,3.952328,"lst, ndvi, discharge, slope, no label, stratified",13 rivers,wt,256,147,8,10,2024-10-31,14:56:12,81.14,Physics-guided


In [25]:
ds = pd.read_excel(f'../results/img_wise_CNN_results.xlsx')
for i,r in ds.iterrows():
    if r['Epochs']==10 and r['Batch size']==16:
        print(list(r))


[3.596295118331909, 'lst, ndvi, no label', '13 rivers', 'wt', 256, 147, 16, 10, '2024-10-30', '16:53:12', 64.91, 'Physics-guided']
[3.606398582458496, 'lst, slope, discharge, no label', '13 rivers', 'wt', 256, 147, 16, 10, '2024-10-30', '17:01:12', 61.02, 'Physics-guided']
[3.610826969146729, 'lst, slope, discharge, ndvi, no label', '13 rivers', 'wt', 256, 147, 16, 10, '2024-10-30', '17:09:33', 65.83, 'Physics-guided']
[3.601340293884277, 'lst, no label', '13 rivers', 'wt', 256, 147, 16, 10, '2024-10-30', '17:17:52', 64.03, 'Physics-guided']
[3.674090147018433, 'lst, ndvi, no label', '13 rivers', 'wt', 256, 147, 16, 10, '2024-10-31', '09:20:38', 65.53, 'Physics-guided']
[3.712287664413452, 'lst, ndvi, label', '13 rivers', 'wt', 256, 147, 16, 10, '2024-10-31', '09:21:40', 62.04, 'Physics-guided']
[3.746084451675415, 'lst, ndvi, no label', '13 rivers', 'wt', 256, 147, 16, 10, '2024-10-31', '09:36:47', 69.57, 'RMSE']
[3.768811941146851, 'lst, ndvi, label', '13 rivers', 'wt', 256, 147, 16,

## Visualize results

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
plt.figure(figsize=(16,5))
#plt.clf
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Simpler CNN MSE Loss during training --- lst+ndvi with wt target')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.savefig('../plots/cnn.png')

In [ ]:
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('MAE during training')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

#### Validate and test

Image wise metrics results

See what are the areas with more prediction error

In [ ]:
if len(validation_prediction.shape) == 4:
  validation_prediction=np.squeeze(validation_prediction, axis=3)

diff = validation_prediction - validation_target
for i in range(diff.shape[0]):
  sns.heatmap(diff[i], cmap='coolwarm')
  plt.title('Prediction Error Heatmap')
  plt.show()

Dispersion graph and histogram of prediction errors

In [ ]:
validation_target_flat = validation_target.reshape(-1)
validation_prediction_flat = validation_prediction.reshape(-1)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Supongamos que y_true son tus valores reales y y_pred son tus predicciones
y_true = validation_target_flat # Valores reales
y_pred = validation_prediction_flat  # Predicciones del modelo

# Visualización
plt.scatter(y_true, y_pred)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'r--')
plt.title('True Values vs Predictions')
plt.show()

# Histograma de errores
errors = y_pred - y_true
plt.hist(errors, bins=30)
plt.xlabel('Error')
plt.ylabel('Frequency')
plt.title('Histogram of Prediction Errors')
plt.show()
